In [1]:
from google.colab import drive
drive.mount('/content/drive/')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive/


In [0]:
!unzip -q drive/'My Drive'/dataset-original.zip

In [0]:
!mkdir dataset-validation
!mkdir dataset-validation/cardboard
!mkdir dataset-validation/glass
!mkdir dataset-validation/paper
!mkdir dataset-validation/plastic
!mkdir dataset-validation/metal
!mkdir dataset-validation/trash
!ls
!ls dataset-original/trash

In [0]:
!shuf -n 101 -e dataset-original/glass/* | xargs -i mv {} dataset-validation/glass/
!shuf -n 144 -e dataset-original/paper/* | xargs -i mv {} dataset-validation/paper/
!shuf -n 53 -e dataset-original/cardboard/* | xargs -i mv {} dataset-validation/cardboard/
!shuf -n 82 -e dataset-original/plastic/* | xargs -i mv {} dataset-validation/plastic/
!shuf -n 60 -e dataset-original/metal/* | xargs -i mv {} dataset-validation/metal/
!shuf -n 37 -e dataset-original/trash/* | xargs -i mv {} dataset-validation/trash/

In [0]:
!ls dataset-original/glass | wc -l
!ls dataset-validation/glass | wc -l

In [0]:
import torch
import torchvision
import torchvision.transforms as transforms
import torchvision.datasets as datasets
import torch.nn as nn
import numpy as np

In [0]:
transformationsTrain = transforms.Compose([transforms.Resize(255), transforms.RandomHorizontalFlip(), transforms.RandomVerticalFlip(),
                                           transforms.RandomRotation(degrees = (90, -90)), transforms.ToTensor(), 
                                           transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))])

transformationsValidation = transforms.Compose([transforms.Resize(255), transforms.ToTensor(), transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))])

In [0]:
train_set = datasets.ImageFolder("dataset-original", transform = transformationsTrain)
val_set = datasets.ImageFolder("dataset-validation", transform = transformationsValidation)

In [0]:
train_loader = torch.utils.data.DataLoader(train_set, batch_size=50, shuffle=True)
val_loader = torch.utils.data.DataLoader(val_set, batch_size =50, shuffle=True)

In [0]:
model = torchvision.models.resnet34(pretrained=True)
model.cuda()

In [0]:
learningRate = 1e-3
epochs = 10
minibatches = 5

In [0]:
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), learningRate)

train_losses, train_accs = [], []
val_losses, val_accs = [], []

for epoch in range(epochs):  # loop over the dataset multiple times

    running_loss = 0.0
    model.train()
    for i, data in enumerate(train_loader, 0):
        # get the inputs; data is a list of [inputs, labels]
        inputs, labels = data
        inputs = inputs.cuda()
        labels = labels.cuda()

        # zero the parameter gradients
        optimizer.zero_grad()

        # forward + backward + optimize
        outputs = model(inputs)

        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        pred = outputs.data.max(1)[1]
        matches = labels == pred
        accuracy = matches.float().mean()

        # print statistics
        running_loss += loss.item()
        if i % minibatches == minibatches - 1:    # print every n mini-batches
            print('[%d, %5d] loss: %.3f accuracy: %.3f itemLoss: %.3f' %
                  (epoch + 1, i + 1, running_loss / minibatches, accuracy, loss.item()))
            running_loss = 0.0

        train_losses.append(loss.item())
        train_accs.append(accuracy.item())
    
    model.eval()
    val_loss, correct = 0., 0
    total = 0
    with torch.no_grad():
      for data in val_loader:
        inputs, labels = data
        inputs = inputs.cuda()
        labels = labels.cuda()
        outputs = model(inputs)
        val_loss += criterion(outputs, labels).item()
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()
    print('Accuracy of the network on the validation images: %d %%' % (
    100 * correct / total))

    val_loss /= len(val_loader)
    acc = correct / len(val_loader)

    val_losses.append(val_loss)
    val_accs.append(acc)

print('Finished Training')